In [1]:
import pandas as pd
import numpy as np
import math as mth
import matplotlib.pyplot as plt
from scipy.integrate import quad

In [2]:
# Define the blocking probability 
def B(a, n):
    Blocking_probability=1
    for i in range(1,n+1):
        Blocking_probability=a * Blocking_probability / (i + a * Blocking_probability)
    return Blocking_probability

In [3]:
# Define the first order derivative of B over a 
def B_prime_a(a, n):
    return ((n / a) - 1 + B(a,n)) * B(a,n)

In [4]:
# Define the first order derivative of B over n 

def B_prime_n(a, n):
    integrand = lambda z: np.exp(-a * z) * (1 + z)**n * np.log(1 + z)
    integral_result, _ = quad(integrand, 0, np.inf)
    return -a * B(a,n)**2 * integral_result


In [5]:
# Define the second order derivative of B over a 

def B_double_prime_a(a, n):
    term1 = (B_prime_a(a,n) - (n / a**2)) * B(a,n)
    term2 = (B_prime_a(a,n)**2) / B(a,n)
    return term1 + term2


In [6]:
# Define the second order derivative of B over n 

def B_double_prime_n(a, n):
    integrand1 = lambda z: np.exp(-a * z) * (1 + z)**n * np.log(1 + z)
    integral1, _ = quad(integrand1, 0, np.inf)
    
    integrand2 = lambda z: np.exp(-a * z) * (1 + z)**n * (np.log(1 + z))**2
    integral2, _ = quad(integrand2, 0, np.inf)
    
    term1 = 2 * a**2 * B(a,n)**3 * (integral1**2)
    term2 = a * B(a,n)**2 * integral2
    return term1 - term2


In [7]:
# Define the second order derivative of B over a and n 

def B_double_prime_a_n(a, n):
    term1 = (n / a - 1 + 2 * B(a,n)) * B_prime_n(a, n)
    term2 = B(a,n) / a
    return term1 + term2

In [8]:
# The first order derivative of a over n

def first_derivation(a,n):
    return -1*B_prime_n(a,n)/B_prime_a(a,n)

In [9]:
# The second order derivative of a over n

def second_derivation(a,n):
    first_term=(B_double_prime_n(a,n)+B_double_prime_a_n(a,n)*first_derivation(a,n))*B_prime_a(a,n)
    second_term=(B_double_prime_a_n(a,n)+B_double_prime_a(a,n)*first_derivation(a,n))*B_prime_n(a,n)
    third_term=first_term-second_term
    return -1*third_term/(B_prime_a(a,n))**2

In [10]:
# Benjamin Proof

def Benjamin_fun(a,n):
    return 2*(B_prime_a(a, n)**2)+B_double_prime_a(a,n)*(1-B(a,n))

In [11]:
a_range=np.linspace(1,200,200)
n_range=np.arange(1,200,1)
results=np.zeros((len(a_range),len(n_range)))
results_benjamin=np.zeros((len(a_range),len(n_range)))

for i_a,a in enumerate(a_range):
    for i_n,n in enumerate(n_range):
        results[i_a,i_n]=second_derivation(a,n)
        results_benjamin[i_a,i_n]=Benjamin_fun(a,n)


C:\Users\mahyar.taheribavi\AppData\Local\Temp\ipykernel_19548\146033194.py:4: RuntimeWarning: overflow encountered in power
  integrand1 = lambda z: np.exp(-a * z) * (1 + z)**n * np.log(1 + z)
C:\Users\mahyar.taheribavi\AppData\Local\Temp\ipykernel_19548\146033194.py:4: RuntimeWarning: invalid value encountered in scalar multiply
  integrand1 = lambda z: np.exp(-a * z) * (1 + z)**n * np.log(1 + z)
C:\Users\mahyar.taheribavi\AppData\Local\Temp\ipykernel_19548\146033194.py:5: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  integral1, _ = quad(integrand1, 0, np.inf)
C:\Users\mahyar.taheribavi\AppData\Local\Temp\ipykernel_19548\146033194.py:7: RuntimeWarning: overflow encountered in power
  integrand2 = lambda z: np.exp(-a * z) * (1 + z)**n * (np.log(1 + z))**2
C:\Users\mahyar.taheribavi\AppData\Local\Temp\ipykernel_19548\146033194.py:7: RuntimeWarning: invalid value enco

In [12]:
# This shows that sign of the second-order derivate is always positive
(results < 0).any().any()

False

In [18]:
import pandas as pd

# Creating the DataFrames
df = pd.DataFrame(results, index=a_range, columns=n_range)
df_benjamin = pd.DataFrame(results_benjamin, index=a_range, columns=n_range)

# Writing the DataFrames to Excel
with pd.ExcelWriter('Test_convexity.xlsx', engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='Results',startrow=1)
    df_benjamin.to_excel(writer, sheet_name='Benjamin',startrow=1)

    # Accessing the workbook and sheets
    workbook = writer.book
    results_sheet = writer.sheets['Results']
    benjamin_sheet = writer.sheets['Benjamin']

    # Adding custom labels using openpyxl
    results_sheet.cell(row=1, column=2, value="n")  # Add "n" at the top-left of the results sheet
    results_sheet.cell(row=2, column=1, value="a")  # Add "a" at the top-left of the results sheet

    benjamin_sheet.cell(row=1, column=2, value="n")  # Add "n" at the top-left of the benjamin sheet
    benjamin_sheet.cell(row=2, column=1, value="a")  # Add "a" at the top-left of the benjamin sheet



In [15]:
df

n,1,2,3,4,5,6,7,8,9,10,...,190,191,192,193,194,195,196,197,198,199
a,,,,,,,,,,,,,,,,,,,,,
1.0,0.435886,0.163911,0.101740,0.074849,0.059450,0.049235,0.041874,0.036291,0.031906,0.028374,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,0.356711,0.129236,0.080181,0.060368,0.049612,0.042616,0.037518,0.033537,0.030296,0.027589,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,0.300834,0.105037,0.063994,0.048127,0.040010,0.035027,0.031529,0.028824,0.026595,0.024686,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,0.260256,0.088027,0.052480,0.039048,0.032449,0.028630,0.026112,0.024256,0.022760,0.021476,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,0.229488,0.075566,0.044104,0.032349,0.026697,0.023557,0.021614,0.020282,0.019271,0.018432,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196.0,0.010001,0.002538,0.001145,0.000654,0.000425,0.000299,0.000223,0.000174,0.000139,0.000115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197.0,0.009951,0.002525,0.001139,0.000650,0.000422,0.000298,0.000222,0.000173,0.000139,0.000114,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198.0,0.009902,0.002512,0.001133,0.000647,0.000420,0.000296,0.000221,0.000172,0.000138,0.000113,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df_benjamin

n,1,2,3,4,5,6,7,8,9,10,...,190,191,192,193,194,195,196,197,198,199
a,,,,,,,,,,,,,,,,,,,,,
1.0,0.000000e+00,6.400000e-02,1.142578e-01,8.214838e-02,3.405302e-02,9.720750e-03,2.117113e-03,3.740907e-04,5.575785e-05,7.197819e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,-3.122502e-17,8.000000e-03,2.886718e-02,4.794299e-02,4.764372e-02,3.189797e-02,1.566757e-02,6.031107e-03,1.910493e-03,5.156118e-04,...,-1.042111e-294,-1.096959e-296,-1.148648e-298,-1.196508e-300,-1.239905e-302,-1.278253e-304,-1.311028e-306,-1.337784e-308,-1.358156e-310,-1.371875e-312
3.0,-2.602085e-17,1.628333e-03,7.510241e-03,1.780711e-02,2.766221e-02,3.050640e-02,2.503303e-02,1.593643e-02,8.190930e-03,3.520586e-03,...,-4.884006e-262,-7.711589e-264,-1.211244e-265,-1.892569e-267,-2.941817e-269,-4.549202e-271,-6.998773e-273,-1.071241e-274,-1.631331e-276,-2.471714e-278
4.0,1.344411e-17,4.551661e-04,2.313424e-03,6.435276e-03,1.256358e-02,1.854540e-02,2.131971e-02,1.945869e-02,1.439283e-02,8.833045e-03,...,-5.533045e-239,-1.164852e-240,-2.439480e-242,-5.082250e-244,-1.053316e-245,-2.171786e-247,-4.454945e-249,-9.091724e-251,-1.846035e-252,-3.729364e-254
5.0,-1.301043e-18,1.579373e-04,8.353459e-04,2.502212e-03,5.490233e-03,9.553495e-03,1.355954e-02,1.586416e-02,1.540583e-02,1.253655e-02,...,-3.370949e-221,-8.870918e-223,-2.322230e-224,-6.047474e-226,-1.566703e-227,-4.037894e-229,-1.035358e-230,-2.641218e-232,-6.703600e-234,-1.692828e-235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196.0,-3.336290e-19,1.368549e-13,6.250776e-13,1.691990e-12,3.578339e-12,6.538992e-12,1.084361e-11,1.677743e-11,2.464195e-11,3.475586e-11,...,4.876870e-05,5.146634e-05,5.424682e-05,5.710147e-05,6.001955e-05,6.298815e-05,6.599201e-05,6.901343e-05,7.203219e-05,7.502553e-05
197.0,-8.829222e-20,1.327593e-13,6.063312e-13,1.641117e-12,3.470496e-12,6.341439e-12,1.051521e-11,1.626807e-11,2.389197e-11,3.369543e-11,...,4.588477e-05,4.846819e-05,5.114001e-05,5.389349e-05,5.672005e-05,5.960911e-05,6.254792e-05,6.552144e-05,6.851223e-05,7.150039e-05
198.0,5.462453e-20,1.288063e-13,5.882343e-13,1.592024e-12,3.366426e-12,6.150816e-12,1.019836e-11,1.577666e-11,2.316848e-11,3.267254e-11,...,4.314458e-05,4.561160e-05,4.817081e-05,5.081718e-05,5.354405e-05,5.634293e-05,5.920339e-05,6.211285e-05,6.505650e-05,6.801713e-05


In [17]:
df

n,1,2,3,4,5,6,7,8,9,10,...,190,191,192,193,194,195,196,197,198,199
a,,,,,,,,,,,,,,,,,,,,,
1.0,0.435886,0.163911,0.101740,0.074849,0.059450,0.049235,0.041874,0.036291,0.031906,0.028374,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,0.356711,0.129236,0.080181,0.060368,0.049612,0.042616,0.037518,0.033537,0.030296,0.027589,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,0.300834,0.105037,0.063994,0.048127,0.040010,0.035027,0.031529,0.028824,0.026595,0.024686,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,0.260256,0.088027,0.052480,0.039048,0.032449,0.028630,0.026112,0.024256,0.022760,0.021476,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,0.229488,0.075566,0.044104,0.032349,0.026697,0.023557,0.021614,0.020282,0.019271,0.018432,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196.0,0.010001,0.002538,0.001145,0.000654,0.000425,0.000299,0.000223,0.000174,0.000139,0.000115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197.0,0.009951,0.002525,0.001139,0.000650,0.000422,0.000298,0.000222,0.000173,0.000139,0.000114,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198.0,0.009902,0.002512,0.001133,0.000647,0.000420,0.000296,0.000221,0.000172,0.000138,0.000113,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
